In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.resolve()))

from pprint import PrettyPrinter
pp = PrettyPrinter()

# Uncomment to get more debugging printouts:
"""
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

In [ ]:
from trulens_eval.keys import *

In [ ]:
from trulens_eval import tru_feedback, Feedback, Query
from trulens_eval.utils.llama import WithFeedbackFilterNodes

import numpy as np

# Construct feedback functions.

hugs = tru_feedback.Huggingface()
openai = tru_feedback.OpenAI()

# Language match between question/answer.
f_lang_match = Feedback(hugs.language_match).on(
    text1=Query.RecordInput, text2=Query.RecordOutput
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on(
    prompt=Query.RecordInput, response=Query.RecordOutput
)

# Question/statement relevance between question and each context chunk.
f_qs_relevance = tru_feedback.Feedback(openai.qs_relevance).on(
    question=Query.RecordInput,
    statement=Query.Record.model.retriever.retrieve.rets[:].node.text
).aggregate(np.min)

feedbacks = [f_lang_match, f_qa_relevance, f_qs_relevance]

In [ ]:
from llama_index import VectorStoreIndex
from llama_index.vector_stores import PineconeVectorStore
from llama_index.indices.vector_store.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
import pinecone

# Pinecone configuration if using pinecone.
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_ENV  # next to api key in console
)

pinecone_index = pinecone.Index(index_name="llmdemo")

pinecone_store = PineconeVectorStore(pinecone_index)
index = VectorStoreIndex.from_vector_store(pinecone_store)
retriever = VectorIndexRetriever(index=index, similarity_top_k=4)
retriever_filtered = WithFeedbackFilterNodes.of_index_retriever(retriever, feedback=f_qs_relevance, threshold = 0.5)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
)

query_engine_filtered = RetrieverQueryEngine(
    retriever=retriever_filtered,
)

In [ ]:
res = query_engine.query("Who is Shayak?")

In [ ]:
pp.pprint(res.response)
pp.pprint(res.source_nodes)

In [ ]:
res = query_engine_filtered.query("Who is Shayak?")

In [ ]:
pp.pprint(res.response)
pp.pprint(res.source_nodes)

In [ ]:
from trulens_eval import Tru

tru = Tru()
Tru().reset_database()

proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent.parent)
thread = Tru().start_evaluator(restart=True)

In [ ]:
l = tru.Llama(engine=query_engine, feedbacks=feedbacks, chain_id="default")
l_filtered = tru.Llama(engine=query_engine_filtered, feedbacks=feedbacks, chain_id="context filtered")

In [ ]:
list(l.instrumented())

In [ ]:
res, record = l.query_with_record("Who is Shayak?")

In [ ]:
# record.dict()

In [ ]:
res, record = l_filtered.query_with_record("Who is Shayak?")